## 라이브러리

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [20]:
import os
path = '../script'

# 다큐멘터리(0), 영화(1), 드라마(2) 선택
category = 0
if category == 0:
    path = path + '/Documentary'
    type = 'Documentary'
elif category == 1:
    path = path + '/Movie'
    type = 'Movie'
elif category == 2:
    path = path + '/Drama'
    type = 'Drama'   

_list = os.listdir(path)
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

file_list = sorted(file_list)
print(f'작품개수: {len(file_list)}')
# print([{i:_list[i]} for i in range(len(file_list))])
for index, title in enumerate(file_list):
    print(f'{index}: {title}')


작품개수: 18
0: 13TH
1: Bo.Burnham_.Inside
2: Bo.Burnham_.Make.Happy
3: Chasing.Coral
4: Dave.Chappelle_.Sticks.&.Stones
5: Dave.Chappelle_.Sticks._.Stones
6: David.Attenborough_.A.Life.on.Our.Planet
7: Disclosure
8: FYRE_.The.Greatest.Party.That.Never.Happened
9: Heroin(e)
10: Inside.Job
11: Kiss.the.Ground
12: Knock.Down.The.House
13: Seaspiracy
14: The.Social.Dilemma
15: Three.Identical.Strangers
16: Virunga
17: What.the.Health


In [21]:
def get_running_time_sec(title, type):

    path = f'../script/{type}'
    file_name = title + '.WEBRip.Netflix.en[cc]'
    if type == "Drama":
        title = title.split('.S0')[0]
    print(f'{path}/{title}/{file_name}')

    # vtt 파일 읽기
    opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
    content = opened_file.read()
    segments = content.split('\n\n')    # 한줄씩 정리

    # 불필요한 부분 제거
    m = re.compile(r"\<.*?\>")
    def clean(content):
        if '♪' in content: return None
        if '[' in content: return None
        if ']' in content: return None
        if '(' in content: return None
        if ')' in content: return None
        if '{' in content: return None
        if '}' in content: return None
        new_content = m.sub('',content)
        new_content = new_content.replace('-->','')
        remove_point_1 = new_content.find('position:')
        remove_point_2 = new_content.find('% \n')
        new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
        return new_content

    new_segments = [clean(s) for s in segments if len(s)!=0][4:]

    # 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리
    trimmed_segments = []
    for segment in new_segments:
        if segment != None and segment.strip() != '':
            split_segment = segment.split()
            times = split_segment[:3]
            script = ' '.join(split_segment[3:])
            times.append(script)
            trimmed_segments.append(times)

    df = pd.DataFrame(columns=['num', 'start', 'end', 'script'])

    for row in trimmed_segments:
        df.loc[row[0]] = row

    df['start'] = pd.to_datetime(df['start'])
    df['end'] = pd.to_datetime(df['end'])

    duration = df['end'][len(df)-1] - df['start'][0]

    return duration.total_seconds()



In [23]:
table = pd.read_csv('../script/csv_files/contents_running_time.csv')

if type == 'Documentary' or type == 'Movie':
    for title in file_list:
        row = []
        row.append(title)
        row.append(get_running_time_sec(title, type))
        row.append(3600/row[1])
        table.loc[title] = row

if type == 'Drama':
    for title in file_list:
        drama_list = os.listdir('../script/Drama/' + title)
        episode_list = []
        for episode in drama_list:
            if '.vtt' in episode:
                episode = episode.split('.WEBRip.')[0]
                episode_list.append(episode)
        
        totol_running_time = 0
        for episode in episode_list:
            totol_running_time += get_running_time_sec(episode, type)
        
        row = []
        row.append(title)
        row.append(totol_running_time)
        row.append(3600/row[1])
        table.loc[title] = row

table

../script/Documentary/13TH/13TH.WEBRip.Netflix.en[cc]
../script/Documentary/Bo.Burnham_.Inside/Bo.Burnham_.Inside.WEBRip.Netflix.en[cc]
../script/Documentary/Bo.Burnham_.Make.Happy/Bo.Burnham_.Make.Happy.WEBRip.Netflix.en[cc]
../script/Documentary/Chasing.Coral/Chasing.Coral.WEBRip.Netflix.en[cc]
../script/Documentary/Dave.Chappelle_.Sticks.&.Stones/Dave.Chappelle_.Sticks.&.Stones.WEBRip.Netflix.en[cc]
../script/Documentary/Dave.Chappelle_.Sticks._.Stones/Dave.Chappelle_.Sticks._.Stones.WEBRip.Netflix.en[cc]
../script/Documentary/David.Attenborough_.A.Life.on.Our.Planet/David.Attenborough_.A.Life.on.Our.Planet.WEBRip.Netflix.en[cc]
../script/Documentary/Disclosure/Disclosure.WEBRip.Netflix.en[cc]
../script/Documentary/FYRE_.The.Greatest.Party.That.Never.Happened/FYRE_.The.Greatest.Party.That.Never.Happened.WEBRip.Netflix.en[cc]
../script/Documentary/Heroin(e)/Heroin(e).WEBRip.Netflix.en[cc]
../script/Documentary/Inside.Job/Inside.Job.WEBRip.Netflix.en[cc]
../script/Documentary/Kiss.the

,title,running_time(sec),weight
0,About.Time,6962.126,0.517083
1,Arthur.Christmas,5404.775,0.666078
2,Baby.Driver,6654.023,0.541026
3,Bad.Boys,6847.716,0.525723
4,Bee.Movie,5425.420,0.663543
...,...,...,...
Seaspiracy,Seaspiracy,5198.359,0.692526
The.Social.Dilemma,The.Social.Dilemma,5546.040,0.649112
Three.Identical.Strangers,Three.Identical.Strangers,5542.412,0.649537
Virunga,Virunga,5492.720,0.655413


In [26]:
table.to_csv('../script/csv_files/contents_running_time.csv', index=False)

In [25]:
hi = pd.read_csv('../script/csv_files/contents_running_time.csv')
hi

,title,running_time(sec),weight
0,About.Time,6962.126,0.517083
1,Arthur.Christmas,5404.775,0.666078
2,Baby.Driver,6654.023,0.541026
3,Bad.Boys,6847.716,0.525723
4,Bee.Movie,5425.420,0.663543
...,...,...,...
68,Seaspiracy,5198.359,0.692526
69,The.Social.Dilemma,5546.040,0.649112
70,Three.Identical.Strangers,5542.412,0.649537
71,Virunga,5492.720,0.655413


In [20]:
_list = os.listdir('../script/Drama')
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

drama_list = os.listdir('../script/Drama/' + file_list[0])
episode_list = []
for episode in _list:
    if '.vtt' in episode:
        episode = episode.split('.WEBRip.')[0]
        episode_list.append(episode)

episode_list


['Disenchantment.S01E10',
 'Disenchantment.S01E05',
 'Disenchantment.S01E02',
 'Disenchantment.S01E07',
 'Disenchantment.S01E08',
 'Disenchantment.S01E09',
 'Disenchantment.S01E01',
 'Disenchantment.S01E06',
 'Disenchantment.S01E03',
 'Disenchantment.S01E04']